In [ ]:
import pandas as pd

# --- 1. CONFIGURATION (The "Market Data") ---
# In a real system, these come from live feeds (e.g., Bloomberg, Refinitiv)

cargo_size_mmbtu = 3_600_000  # Standard large LNG vessel
boil_off_daily = 0.0015       # 0.15% gas lost per day

market_data = {
    'US_Henry_Hub': 3.50,     # Cost of gas in USA ($/MMBtu)
    'Liquefaction_Fee': 2.50, # Tolling fee to turn gas to liquid
    'Europe_TTF': 12.00,      # Market Price in Europe ($/MMBtu)
    'Asia_JKM': 14.50,        # Market Price in Asia ($/MMBtu)
}

# Shipping Assumptions (Spot Charter)
shipping_data = {
    'Charter_Rate_Daily': 85_000,  # $85k per day
    'Fuel_Cost_Daily': 30_000,     # Bunker fuel cost
    'Panama_Canal_Fee': 400_000,   # Flat fee for Asia route
}

# Logistics: Days at sea (One way)
routes = {
    'USGC_to_Europe': {'days': 14, 'canal_fee': 0},
    'USGC_to_Asia':   {'days': 26, 'canal_fee': shipping_data['Panama_Canal_Fee']}
}

# --- 2. THE CALCULATION ENGINE ---

def calculate_economics(destination, price, route_info):
    days = route_info['days']
    canal_fee = route_info['canal_fee']
    
    # A. Calculate Shipping Cost (Round Trip duration usually factored for Charter)
    # We assume round-trip days for charter cost, but one-way for fuel/boil-off
    round_trip_days = days * 2 
    total_charter_cost = round_trip_days * shipping_data['Charter_Rate_Daily']
    total_fuel_cost = round_trip_days * shipping_data['Fuel_Cost_Daily']
    
    total_shipping_cost = total_charter_cost + total_fuel_cost + canal_fee
    
    # B. Calculate Boil-Off (Gas lost during transit)
    # The cargo shrinks while sailing
    volume_loss_pct = 1 - ((1 - boil_off_daily) ** days)
    volume_delivered = cargo_size_mmbtu * (1 - volume_loss_pct)
    volume_lost = cargo_size_mmbtu - volume_delivered
    
    # C. Unit Freight Cost ($/MMBtu)
    # We spread the total shipping cost over the delivered volume
    freight_per_mmbtu = total_shipping_cost / volume_delivered
    
    # D. The Netback Calculation
    netback = price - freight_per_mmbtu
    
    # E. The Margin (Profit)
    # Cost to Load = Henry Hub + Liquefaction Fee
    cost_fob = market_data['US_Henry_Hub'] + market_data['Liquefaction_Fee']
    margin = netback - cost_fob
    
    return {
        'Destination': destination,
        'Market_Price': f"${price:.2f}",
        'freight_unit_cost': freight_per_mmbtu,
        'Netback_at_USGC': netback,
        'Final_Margin': margin,
        'Volume_Lost_MMBtu': int(volume_lost)
    }

# --- 3. RUNNING THE COMPARISON ---

results = []
results.append(calculate_economics('Europe (TTF)', market_data['Europe_TTF'], routes['USGC_to_Europe']))
results.append(calculate_economics('Asia (JKM)', market_data['Asia_JKM'], routes['USGC_to_Asia']))

# Create DataFrame for the Dashboard
df = pd.DataFrame(results)

# Formatting for display
pd.options.display.float_format = '${:,.2f}'.format
df = df.set_index('Destination')

print("--- LNG TRADING DESK: ARBITRAGE MONITOR ---")
print(f"US Gas Cost (FOB): ${market_data['US_Henry_Hub'] + market_data['Liquefaction_Fee']:.2f} / MMBtu")
print("-" * 50)
print(df[['Market_Price', 'freight_unit_cost', 'Netback_at_USGC', 'Final_Margin']])